In [13]:
"""
Sequence-based structural alignment of two proteins.
"""

from __future__ import print_function, division

import argparse
import os

from Bio.PDB import PDBParser, FastMMCIFParser, Superimposer, PDBIO
from Bio.PDB.Polypeptide import is_aa

from Bio import pairwise2
from Bio.SubsMat import MatrixInfo as matlist
from Bio.Data.SCOPData import protein_letters_3to1 as aa3to1

In [14]:
def align_sequences(structA, structB, **kwargs):
    """
    Performs a global pairwise alignment between two sequences
    using the BLOSUM62 matrix and the Needleman-Wunsch algorithm
    as implemented in Biopython. Returns the alignment, the sequence
    identity and the residue mapping between both original sequences.
    """

    def _calculate_identity(sequenceA, sequenceB):
        """
        Returns the percentage of identical characters between two sequences.
        Assumes the sequences are aligned.
        """

        sa, sb, sl = sequenceA, sequenceB, len(sequenceA)
        matches = [sa[i] == sb[i] for i in range(sl)]
        seq_id = (100 * sum(matches)) / sl

        gapless_sl = sum([1 for i in range(sl) if (sa[i] != '-' and sb[i] != '-')])
        gap_id = (100 * sum(matches)) / gapless_sl
        return (seq_id, gap_id)

    def _get_pdb_sequence(structure):
        """
        Retrieves the AA sequence from a PDB structure.
        """

        _aainfo = lambda r: (r.id[1], aa3to1.get(r.resname, 'X'))
        seq = [_aainfo(r) for r in structure.get_residues() if is_aa(r)]
        return seq

    matrix = kwargs.get('matrix', matlist.blosum62)
    gap_open = kwargs.get('gap_open', -10.0)
    gap_extend = kwargs.get('gap_extend', -0.5)
    trim_ends = kwargs.get('trim_ends', True)

    resseq_A = _get_pdb_sequence(structA)
    resseq_B = _get_pdb_sequence(structB)

    sequence_A = ''.join([i[1] for i in resseq_A])
    sequence_B = ''.join([i[1] for i in resseq_B])
    alns = pairwise2.align.globalds(sequence_A, sequence_B,
                                    matrix, gap_open, gap_extend,
                                    penalize_end_gaps=(False, False) )

    best_aln = alns[0]
    aligned_A, aligned_B, score, begin, end = best_aln

    # Equivalent residue numbering
    # Relative to reference
    mapping = {}
    aa_i_A, aa_i_B = 0, 0
    for aln_i, (aa_aln_A, aa_aln_B) in enumerate(zip(aligned_A, aligned_B)):
        if aa_aln_A == '-':
            if aa_aln_B != '-':
                aa_i_B += 1
        elif aa_aln_B == '-':
            if aa_aln_A != '-':
                aa_i_A += 1
        else:
            assert resseq_A[aa_i_A][1] == aa_aln_A
            assert resseq_B[aa_i_B][1] == aa_aln_B
            mapping[resseq_A[aa_i_A][0]] = resseq_B[aa_i_B][0]
            aa_i_A += 1
            aa_i_B += 1

  
    # Calculate (gapless) sequence identity
    seq_id, g_seq_id = _calculate_identity(aligned_A, aligned_B)
    return ((aligned_A, aligned_B), seq_id, g_seq_id, mapping)
    # return ((trim_aln_A, trim_aln_B, mismatch), seq_id, g_seq_id, mapping)

pdbparser = PDBParser()
structure_1 = pdbparser.get_structure("6N4Q", "../DATA/SUBUNITS/SCN/6N4Q.pdb")
print(structure_1[0])

<Model id=0>


In [15]:
#for SCN protein alignment
# pdbparser = PDBParser()
    
# structure_1 = pdbparser.get_structure("6AGF", "../DATA/SUBUNITS/SCN/6AGF.pdb")
# chain_1 = structure_1[0]['A']

# structure_2 = pdbparser.get_structure("6J8E", "../DATA/SUBUNITS/SCN/6J8E.pdb")
# chain_2 = structure_2[0]['A']

# aln, seq_id, gapless_id, res_map = align_sequences(chain_1, chain_2)

# refe_ca_list, mobi_ca_list = [], []
# for refe_res in res_map:
#     refe_ca_list.append(chain_1[refe_res]['CA'])
#     mobi_ca_list.append(chain_2[res_map[refe_res]]['CA'])

# # Superimpose matching residues
# si = Superimposer()
# si.set_atoms(refe_ca_list, mobi_ca_list)

# # Transform & Write Mobile
# si.apply(chain_2.get_atoms())

# io = PDBIO()
# io.set_structure(chain_2)
# m_transformed_name = '{0}_transformed.pdb'.format(structure_2.id)
# io.save(m_transformed_name)



In [16]:
# align SCN group 1
import yaml

ref_structure_name = '7DTC'

with open('../CONFIG/config_data.yaml', 'r') as f:
    pdb_ids = yaml.load(f, Loader=yaml.FullLoader)

structures = list()

for gene in pdb_ids['SCN']:
    for structure in list(gene.values())[0]:
        structures.append(structure)

# wont_align = ['6N4R', '6N4Q', '6VXO', '7K48', '7K18']
# for s in wont_align:
#     structures.pop(structures.index(s))

ref_structure = pdbparser.get_structure(ref_structure_name, "../DATA/SUBUNITS/SCN/" + ref_structure_name + '.pdb')
# chain_3 = structure_3[0]['A']
io = PDBIO()
for structure_name in structures:
    # os.remove(transformed_name)
    si = Superimposer()
    structure = pdbparser.get_structure(structure_name, "../DATA/SUBUNITS/SCN/" + structure_name + ".pdb")
    # for chain_1, chain in zip(ref_structure.get_chains(), structure.get_chains()): 
    for chain_1 in ref_structure.get_chains():
        refe_ca_list, mobi_ca_list = [], []
        for chain_2 in structure.get_chains():
            aln, seq_id, gapless_id, res_map = align_sequences(chain_1, chain_2)
            for refe_res in res_map:
                refe_ca_list.append(chain_1[refe_res]['CA'])
                mobi_ca_list.append(chain_2[res_map[refe_res]]['CA'])

        # Superimpose matching residues
        if refe_ca_list and mobi_ca_list:
            si.set_atoms(refe_ca_list, mobi_ca_list)

        # Transform & Write Mobile
    si.apply(structure.get_atoms())

        
    io.set_structure(structure)
    transformed_name = structure_name + '.pdb'
    io.save('../DATA/ALIGNED/SCN/' + transformed_name)
    ref_structure = pdbparser.get_structure(ref_structure_name, "../DATA/SUBUNITS/SCN/" + ref_structure_name + '.pdb')
    


In [24]:
# align SCN group 2
# can't get these structures to align for some reason
ref_structure_name = '6N4R'

structures = ['6N4R', '6N4Q', '6VXO']


ref_structure = pdbparser.get_structure(ref_structure_name, "../DATA/ALIGNED/SCN/" + ref_structure_name + '.pdb')
# chain_3 = structure_3[0]['A']
io = PDBIO()
for structure_name in structures:
    # os.remove(transformed_name)
    si = Superimposer()
    structure = pdbparser.get_structure(structure_name, "../DATA/SUBUNITS/SCN/" + structure_name + ".pdb")
    # for chain_1, chain in zip(ref_structure.get_chains(), structure.get_chains()): 
    for chain_1 in ref_structure.get_chains():
        refe_ca_list, mobi_ca_list = [], []
        for chain_2 in structure.get_chains():
            aln, seq_id, gapless_id, res_map = align_sequences(chain_1, chain_2)
            for refe_res in res_map:
                refe_ca_list.append(chain_1[refe_res]['CA'])
                mobi_ca_list.append(chain_2[res_map[refe_res]]['CA'])

        # Superimpose matching residues
        if refe_ca_list and mobi_ca_list:
            si.set_atoms(refe_ca_list, mobi_ca_list)

        # Transform & Write Mobile
    si.apply(structure.get_atoms())

        
    io.set_structure(structure)
    transformed_name = structure_name + '_transformed.pdb'
    io.save('../DATA/ALIGNED/SCN/' + transformed_name)
    ref_structure = pdbparser.get_structure(ref_structure_name, "../DATA/ALIGNED/SCN/" + ref_structure_name + '.pdb')
    

In [26]:


ref_structure_name = '7K48'

structures = ['7K18']


ref_structure = pdbparser.get_structure(ref_structure_name, "../DATA/ALIGNED/SCN/" + ref_structure_name + '.pdb')
# chain_3 = structure_3[0]['A']
io = PDBIO()
for structure_name in structures:
    # os.remove(transformed_name)
    si = Superimposer()
    structure = pdbparser.get_structure(structure_name, "../DATA/SUBUNITS/SCN/" + structure_name + ".pdb")
    # for chain_1, chain in zip(ref_structure.get_chains(), structure.get_chains()): 
    for chain_1 in ref_structure.get_chains():
        refe_ca_list, mobi_ca_list = [], []
        for chain_2 in structure.get_chains():
            aln, seq_id, gapless_id, res_map = align_sequences(chain_1, chain_2)
            for refe_res in res_map:
                refe_ca_list.append(chain_1[refe_res]['CA'])
                mobi_ca_list.append(chain_2[res_map[refe_res]]['CA'])

        # Superimpose matching residues
        if refe_ca_list and mobi_ca_list:
            si.set_atoms(refe_ca_list, mobi_ca_list)

        # Transform & Write Mobile
    si.apply(structure.get_atoms())

        
    io.set_structure(structure)
    transformed_name = structure_name + '.pdb'
    io.save('../DATA/ALIGNED/SCN/' + transformed_name)
    ref_structure = pdbparser.get_structure(ref_structure_name, "../DATA/ALIGNED/SCN/" + ref_structure_name + '.pdb')


In [18]:
# align KCN group 1
import yaml

ref_structure_name = '7EJ1'

with open('../CONFIG/config_data.yaml', 'r') as f:
    pdb_ids = yaml.load(f, Loader=yaml.FullLoader)

structures = list()

for gene in pdb_ids['KCN']:
    for structure in list(gene.values())[0]:
        structures.append(structure)


ref_structure = pdbparser.get_structure(ref_structure_name, "../DATA/SUBUNITS/KCN/" + ref_structure_name + '.pdb')
# chain_3 = structure_3[0]['A']
io = PDBIO()
for structure_name in structures:
    # os.remove(transformed_name)
    si = Superimposer()
    structure = pdbparser.get_structure(structure_name, "../DATA/SUBUNITS/KCN/" + structure_name + ".pdb")
    # for chain_1, chain in zip(ref_structure.get_chains(), structure.get_chains()): 
    for chain_1 in ref_structure.get_chains():
        refe_ca_list, mobi_ca_list = [], []
        for chain_2 in structure.get_chains():
            aln, seq_id, gapless_id, res_map = align_sequences(chain_1, chain_2)
            for refe_res in res_map:
                refe_ca_list.append(chain_1[refe_res]['CA'])
                mobi_ca_list.append(chain_2[res_map[refe_res]]['CA'])

        # Superimpose matching residues
        if refe_ca_list and mobi_ca_list:
            si.set_atoms(refe_ca_list, mobi_ca_list)

        # Transform & Write Mobile
    si.apply(structure.get_atoms())

        
    io.set_structure(structure)
    transformed_name = structure_name + '.pdb'
    io.save('../DATA/ALIGNED/KCN/' + transformed_name)
    ref_structure = pdbparser.get_structure(ref_structure_name, "../DATA/SUBUNITS/KCN/" + ref_structure_name + '.pdb')

In [19]:
# align KCN group 2
import yaml

ref_structure_name = '6V01_transformed'
structures = ['6EBM', '6UZZ', '6V00', '6V01']


ref_structure = pdbparser.get_structure(ref_structure_name, "../DATA/ALIGNED/KCN/" + ref_structure_name + '.pdb')
# chain_3 = structure_3[0]['A']
io = PDBIO()
for structure_name in structures:
    # os.remove(transformed_name)
    si = Superimposer()
    structure = pdbparser.get_structure(structure_name, "../DATA/SUBUNITS/KCN/" + structure_name + ".pdb")
    # for chain_1, chain in zip(ref_structure.get_chains(), structure.get_chains()): 
    for chain_1 in ref_structure.get_chains():
        refe_ca_list, mobi_ca_list = [], []
        for chain_2 in structure.get_chains():
            aln, seq_id, gapless_id, res_map = align_sequences(chain_1, chain_2)
            for refe_res in res_map:
                refe_ca_list.append(chain_1[refe_res]['CA'])
                mobi_ca_list.append(chain_2[res_map[refe_res]]['CA'])

        # Superimpose matching residues
        if refe_ca_list and mobi_ca_list:
            si.set_atoms(refe_ca_list, mobi_ca_list)

        # Transform & Write Mobile
    si.apply(structure.get_atoms())

        
    io.set_structure(structure)
    transformed_name = structure_name + '_transformed.pdb'
    io.save('../DATA/ALIGNED/KCN/' + transformed_name)
    ref_structure = pdbparser.get_structure(ref_structure_name, "../DATA/ALIGNED/KCN/" + ref_structure_name + '.pdb')

FileNotFoundError: [Errno 2] No such file or directory: '../DATA/ALIGNED/KCN/6V01_transformed.pdb'

In [ ]:
# structures = ["6EBK", "7EJ1", "7WF4", "7WF3"]


# m_transformed_name_2 = m_transformed_name
# structure_outer = pdbparser.get_structure("7EJ2_transformed", "../UTIL/" + m_transformed_name_2)
# io = PDBIO()
# for structure_name in structures:
#     os.remove(m_transformed_name_2)
#     si = Superimposer()
#     structure = pdbparser.get_structure(structure_name, "../DATA/SUBUNITS/KCN/" + structure_name + ".pdb")
#     # for chain_1, chain in zip(structure_outer.get_chains(), structure.get_chains()): 
#     for chain_1 in structure_outer.get_chains():
#         refe_ca_list, mobi_ca_list = [], []
#         for chain_2 in structure.get_chains():
#             aln, seq_id, gapless_id, res_map = align_sequences(chain_1, chain_2)
#             for refe_res in res_map:
#                 refe_ca_list.append(chain_1[refe_res]['CA'])
#                 mobi_ca_list.append(chain_2[res_map[refe_res]]['CA'])

#         # Superimpose matching residues
#         if refe_ca_list and mobi_ca_list:
#             si.set_atoms(refe_ca_list, mobi_ca_list)

#         # Transform & Write Mobile
#     si.apply(structure.get_atoms())

        
#     io.set_structure(structure)
#     m_transformed_name_2 = '{0}_transformed.pdb'.format(structure.id)
#     io.save(m_transformed_name_2)
#     structure_outer = pdbparser.get_structure(structure.id, "../UTIL/" + m_transformed_name_2)